In [1]:
import pandas as pd
from temporal_builder import TemporalDataBuilder

In [2]:
tbd = TemporalDataBuilder(data_dir='..\data')

In [3]:
data = tbd.dataset
data

uniqueid,1.0-113.0-3.0,1.0-73.0-23.0,10.0-1.0-2.0,10.0-3.0-1007.0,10.0-3.0-1008.0,10.0-3.0-2004.0,10.0-5.0-1002.0,11.0-1.0-41.0,11.0-1.0-43.0,11.0-1.0-51.0,...,80.0-2.0-12.0,80.0-2.0-14.0,9.0-1.0-10.0,9.0-1.0-1123.0,9.0-11.0-124.0,9.0-3.0-1003.0,9.0-3.0-25.0,9.0-5.0-5.0,9.0-9.0-2123.0,9.0-9.0-27.0
,PM25,PM25,PM25,PM25,PM25,PM25,PM25,PM25,PM25,PM25,...,PM25,PM25,PM25,PM25,PM25,PM25,PM25,PM25,PM25,PM25
DateTime,,,,,,,,,,,,,,,,,,,,,
2018-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 05:00:00,NaN,NaN,5.4,NaN,10.0,7.2,5.2,10.0,12.0,8.0,...,NaN,NaN,11.0,9.0,7.0,6.0,8.0,7.0,0.0,3.0
2018-01-01 06:00:00,16.0,13.4,5.2,NaN,12.1,5.6,5.2,14.0,11.0,12.0,...,NaN,NaN,8.0,9.0,7.0,6.0,10.0,6.0,1.0,3.0
2018-01-01 07:00:00,15.0,21.1,8.0,NaN,12.8,5.1,5.2,11.0,11.0,8.0,...,NaN,NaN,9.0,5.0,6.0,8.0,7.0,5.0,4.0,3.0
2018-01-01 08:00:00,15.0,17.4,7.7,NaN,12.5,4.8,6.1,16.0,9.0,10.0,...,344.0,622.0,8.0,3.0,3.0,7.0,3.0,1.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01 05:00:00,20.0,4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
data

uniqueid,1.0-113.0-3.0,1.0-73.0-23.0,10.0-1.0-2.0,10.0-3.0-1007.0,10.0-3.0-1008.0,10.0-3.0-2004.0,10.0-5.0-1002.0,11.0-1.0-41.0,11.0-1.0-43.0,11.0-1.0-51.0,...,80.0-2.0-12.0,80.0-2.0-14.0,9.0-1.0-10.0,9.0-1.0-1123.0,9.0-11.0-124.0,9.0-3.0-1003.0,9.0-3.0-25.0,9.0-5.0-5.0,9.0-9.0-2123.0,9.0-9.0-27.0
,PM25,PM25,PM25,PM25,PM25,PM25,PM25,PM25,PM25,PM25,...,PM25,PM25,PM25,PM25,PM25,PM25,PM25,PM25,PM25,PM25
DateTime,,,,,,,,,,,,,,,,,,,,,
2018-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 05:00:00,NaN,NaN,5.4,NaN,10.0,7.2,5.2,10.0,12.0,8.0,...,NaN,NaN,11.0,9.0,7.0,6.0,8.0,7.0,0.0,3.0
2018-01-01 06:00:00,16.0,13.4,5.2,NaN,12.1,5.6,5.2,14.0,11.0,12.0,...,NaN,NaN,8.0,9.0,7.0,6.0,10.0,6.0,1.0,3.0
2018-01-01 07:00:00,15.0,21.1,8.0,NaN,12.8,5.1,5.2,11.0,11.0,8.0,...,NaN,NaN,9.0,5.0,6.0,8.0,7.0,5.0,4.0,3.0
2018-01-01 08:00:00,15.0,17.4,7.7,NaN,12.5,4.8,6.1,16.0,9.0,10.0,...,344.0,622.0,8.0,3.0,3.0,7.0,3.0,1.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01 05:00:00,20.0,4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
prova = pd.read_csv('..\data\\aqs_sites.csv')

In [ ]:
sites_red = sites_full.loc[:, "State Code":"Longitude"]
sites_red.insert(0, 'ID', value= None)
sites_red.insert(0, 'Index', value=[i for i in range(sites_red.shape[0])])

sites_red = generate_id(sites_red)
sites_red.drop(columns=['State Code', 'County Code', 'Site Number'], inplace=True)
sites_red = sites_red.loc[nodes_to_keep, :]

In [37]:
data.loc[ 
    : ,
    (['1.0-113.0-3.0','10.0-1.0-2.0'], 'PM25')
]

uniqueid,1.0-113.0-3.0,10.0-1.0-2.0
,PM25,PM25
DateTime,,
2018-01-01 04:00:00,NaN,NaN
2018-01-01 05:00:00,NaN,5.4
2018-01-01 06:00:00,16.0,5.2
2018-01-01 07:00:00,15.0,8.0
2018-01-01 08:00:00,15.0,7.7
...,...,...
2020-01-01 05:00:00,20.0,NaN
2020-01-01 06:00:00,NaN,NaN


In [10]:
def node_info(df, node):
    #print(f'Node: {node}')
    # co = df.loc[:, node]['CO']
    # no2 = df.loc[:, node]['NO2']
    # pm10 = df.loc[:, node]['PM10']
    pm25 = df.loc[:, node]['PM25']
    # so2 = df.loc[:, node]['SO2']

    # co_nans = co.isna().sum()
    # no2_nans = no2.isna().sum()
    # pm10_nans = pm10.isna().sum()
    pm25_nans = pm25.isna().sum()
    # so2_nans = so2.isna().sum()

    # co_perc = round((co_nans/len(co))*100, 2)
    # no2_perc = round((no2_nans/len(no2))*100, 2)
    # pm10_perc = round((pm10_nans/len(pm10))*100, 2)
    pm25_perc = round((pm25_nans/len(pm25))*100, 2)
    # so2_perc = round((so2_nans/len(so2))*100, 2)

    # features = [co_perc, no2_perc, pm10_perc, so2_perc]

    # print(f'CO NaNs: \t{co_nans}\t({co_perc}% of total for this node)')
    # print(f'NO2 NaNs: \t{no2_nans}\t({no2_perc}% of total for this node)')
    # print(f'PM10 NaNs: \t{pm10_nans}\t({pm10_perc}% of total for this node)')
    # print(f'PM25 NaNs: \t{pm25_nans}\t({pm25_perc}% of total for this node)')
    # print(f'SO2 NaNs: \t{so2_nans}\t({so2_perc}% of total for this node)')
    # print()

    if pm25_perc < 50:
        keep = True
    else:
        keep = False

    return keep


In [11]:

# features = ['CO', 'NO2', 'PM10', 'SO2']

# for feature in range(4):
nodes_to_keep = []

for node in data.columns.get_level_values(0).drop_duplicates():
    keep = node_info(data, node)
    if keep: nodes_to_keep.append(node)

print(f'{len(nodes_to_keep)}')


583


In [17]:
print(f'Keep {len(nodes_to_keep)} on {data.shape[1]//5}')

Keep 31 on 665


In [24]:
data.drop(columns=['CO','SO2','NO2','PM10'])

KeyError: 'CO'

In [12]:
from airquality import AirQuality

aq = AirQuality()


download: 5.96MB [00:00, 6.94MB/s]                            


Sites list download completed!
Preparing sites data... Generating IDs for nodes... 


100%|█████████▉| 20753/20757 [00:00<00:00, 48317.73it/s]


ValueError: could not convert string to float: 'CC'